* https://www.tensorflow.org/guide/keras/custom_layers_and_models
* https://www.tensorflow.org/tutorials/customization/custom_layers

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Layer, Input

print(tf.__version__)

2.0.0


* The best way to implement custom layer is extending the `tf.keras.layers.Layer` class.
* The custom layer should implement the following methods:
 * `__init__(**kwargs)`: Save configuration in member variables. Note that the `__init__` method of the base Layer class takes some keyword arguments, in particular a `name` and a `dtype`. It's good practice to pass these arguments to the parent class in `__init__` and to include them in the layer config
 
 * `build(input_shape)`: Called once from `__call__`, when we know the shapes of inputs and `dtype`. Should have the calls to `add_weight()`, and then call the super's `build()` (which sets `self.built = True`, which is nice in case the user wants to call `build()` manually before the first `__call__`).
 
 * `call(inputs, **kwargs)`: Called in `__call__` after making sure `build()` has been called once. Should actually perform the logic of applying the layer to the input tensors (which should be passed in as the first argument). 

## A simple dense layer

In [2]:
class MyDenseLayer(Layer):
    def __init__(self, num_outputs, **kwargs):
        super(MyDenseLayer, self).__init__(**kwargs)
        self.num_outputs = num_outputs
    
    def build(self, input_shape):
        self.kernel = self.add_variable("kernel", shape=[int(input_shape[-1]), self.num_outputs])
        super(MyDenseLayer, self).build(input_shape)  # Be sure to call this at the end
        
    def call(self, inputs):
        return tf.matmul(inputs, self.kernel)

## Composing layers

In [3]:
class DenseBlock(Layer):
    def __init__(self, **kwargs):
        super(DenseBlock, self).__init__(**kwargs)
        self.dense_1 = MyDenseLayer(32)
        self.dense_2 = MyDenseLayer(64)
        self.dense_3 = MyDenseLayer(3)
    def call(self, inputs):
        x = self.dense_1(inputs)
        x = tf.nn.relu(x)
        x = self.dense_2(x)
        x = tf.nn.relu(x)
        return self.dense_3(x)

In [4]:
inputs = Input(shape=(10,))
x = DenseBlock()(inputs)

W1123 14:02:30.932218 20576 deprecation.py:323] From <ipython-input-2-37c6602cccc5>:7: Layer.add_variable (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.add_weight` method instead.


In [5]:
model = Model(inputs, x)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 10)]              0         
_________________________________________________________________
dense_block (DenseBlock)     (None, 3)                 2560      
Total params: 2,560
Trainable params: 2,560
Non-trainable params: 0
_________________________________________________________________
